# 『5과목』 AI와 딥러닝

## 강화 학습과 게임 지능

## Set Up

In [1]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # Windows
# matplotlib.rcParams['font.family'] = 'AppleGothic' # Mac
matplotlib.rcParams['font.size'] = 15 # 글자 크기
matplotlib.rcParams['axes.unicode_minus'] = False # 한글 폰트 사용 시, 마이너스 글자가 깨지는 현상을 해결

### 다중 손잡이 밴딧 문제를 위한 랜덤 정책 프로그램

In [2]:
import numpy as np

# 다중 손잡이 밴딧 문제 설정
arms_profit=[0.4, 0.12, 0.52, 0.6, 0.25]
n_arms=len(arms_profit)

n_trial=10000 # 손잡이를 당기는 횟수(에피소드 길이)

# 손잡이 당기는 행위를 시뮬레이션하는 함수(handle은 손잡이 번호)
def pull_bandit(handle):
    q=np.random.random()
    if q<arms_profit[handle]:
        return 1
    else:
        return -1

# 랜덤 정책을 모방하는 함수
def random_exploration():
    episode=[]
    num=np.zeros(n_arms) # 손잡이별로 당긴 횟수
    wins=np.zeros(n_arms) # 손잡이별로 승리 횟수
    for i in range(n_trial):
        h=np.random.randint(0,n_arms)
        reward=pull_bandit(h)
        episode.append([h,reward])
        num[h]+=1
        wins[h]+=1 if reward==1 else 0
    return episode, (num,wins)

e,r=random_exploration()

print("손잡이별 승리 확률:", ["%6.4f"% (r[1][i]/r[0][i]) if r[0][i]>0 else 0.0 for i in range(n_arms)])
print("손잡이별 수익($):",["%d"% (2*r[1][i]-r[0][i]) for i in range(n_arms)])
print("순 수익($):",sum(np.asarray(e)[:,1]))

손잡이별 승리 확률: ['0.3885', '0.1209', '0.5258', '0.5925', '0.2494']
손잡이별 수익($): ['-427', '-1530', '103', '373', '-1029']
순 수익($): -2510


### ε-탐욕 알고리즘

In [3]:
# ε-탐욕을 구현하는 함수
def epsilon_greedy(eps):
    episode=[]
    num=np.zeros(n_arms) # 손잡이별로 당긴 횟수
    wins=np.zeros(n_arms) # 손잡이별로 승리 횟수
    for i in range(n_trial):
        r=np.random.random()
        if(r<eps or sum(wins)==0): # 확률 eps로 임의 선택
            h=np.random.randint(0,n_arms)
        else:
            prob=np.asarray([wins[i]/num[i] if num[i]>0 else 0.0 for i in range(n_arms)])
            prob=prob/sum(prob)
            h=np.random.choice(range(n_arms),p=prob)
        reward=pull_bandit(h)
        episode.append([h,reward])
        num[h]+=1
        wins[h]+=1 if reward==1 else 0
    return episode, (num,wins)

e,r=epsilon_greedy(0.1)

print("손잡이별 승리 확률:", ["%6.4f"% (r[1][i]/r[0][i]) if r[0][i]>0 else 0.0 for i in range(n_arms)])
print("손잡이별 수익($):",["%d"% (2*r[1][i]-r[0][i]) for i in range(n_arms)])
print("순 수익($):",sum(np.asarray(e)[:,1]))

손잡이별 승리 확률: ['0.4092', '0.1247', '0.5371', '0.5902', '0.2560']
손잡이별 수익($): ['-393', '-566', '203', '542', '-654']
순 수익($): -868


In [8]:
import gym
print(gym.__version__)

0.25.2


In [9]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

DeprecatedEnv: Environment version v0 for `FrozenLake` is deprecated. Please use `FrozenLake-v1` instead.

In [10]:
import gym
# 환경 불러오기
env=gym.make("FrozenLake-v1",is_slippery=False)
print(env.observation_space)
print(env.action_space)

n_trial=20
# 에피소드 수집
env.reset()
episode=[]
for i in range(n_trial):
    action = env.action_space.sample() # 행동을 취함(랜덤 선택)
    step_result = env.step(action) # 보상을 받고 상태가 바뀜
    obs = step_result[0]  # observation 값
    reward = step_result[1]  # reward 값
    done = step_result[2]  # done 값
    info = step_result[3]  # info 값
    episode.append([action, reward, obs, done, info])  # 모든 값을 추가
    # env.render() # 렌더링
    if done:
        break

print(episode)
env.close()

Discrete(16)
Discrete(4)


AttributeError: module 'numpy' has no attribute 'bool8'

### 가치 반복 (Value Iteration) 알고리즘

In [11]:
import numpy as np

# 4x4 FrozenLake 환경에서 상태와 행동의 개수
num_states = 16
num_actions = 4

# 전이 확률 초기화
T = np.zeros((num_states, num_actions, num_states))

# 보상 초기화
R = np.zeros((num_states, num_actions, num_states))

# 상태 전이 예시 (단순화된 형태로 일부만 보여줌)
# 상(0), 하(1), 좌(2), 우(3) 행동에 대한 전이 확률 설정
# 상(0) 행동
T[0, 0, 0] = 1  # 벽이므로 상태가 변하지 않음
T[1, 0, 1] = 1  # 벽이므로 상태가 변하지 않음
T[4, 0, 0] = 1  # (1, 0) -> (0, 0)
T[5, 0, 1] = 1  # (1, 1) -> (0, 1)

# 하(1) 행동
T[0, 1, 4] = 1  # (0, 0) -> (1, 0)
T[1, 1, 5] = 1  # (0, 1) -> (1, 1)
T[4, 1, 8] = 1  # (1, 0) -> (2, 0)
T[5, 1, 9] = 1  # (1, 1) -> (2, 1)

# 좌(2) 행동
T[0, 2, 0] = 1  # 벽이므로 상태가 변하지 않음
T[1, 2, 0] = 1  # (0, 1) -> (0, 0)
T[4, 2, 4] = 1  # 벽이므로 상태가 변하지 않음
T[5, 2, 4] = 1  # (1, 1) -> (1, 0)

# 우(3) 행동
T[0, 3, 1] = 1  # (0, 0) -> (0, 1)
T[1, 3, 2] = 1  # (0, 1) -> (0, 2)
T[4, 3, 5] = 1  # (1, 0) -> (1, 1)
T[5, 3, 6] = 1  # (1, 1) -> (1, 2)

# 보상 설정 예시
R[0, 1, 4] = 0  # (0, 0)에서 하로 이동하여 (1, 0)이 됨 -> 보상 0
R[0, 3, 1] = 0  # (0, 0)에서 우로 이동하여 (0, 1)이 됨 -> 보상 0
R[14, 1, 15] = 1  # 목표 지점에 도달하면 보상 1

print("전이 확률 T:")
print(T)

print("보상 R:")
print(R)

전이 확률 T:
[[[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
보상 R:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 

In [12]:
import numpy as np

# 환경 설정
states = range(16)  # 16개의 상태 (예: FrozenLake 4x4)
actions = range(4)  # 4개의 행동 (상, 하, 좌, 우)
gamma = 0.9  # 할인 인자
epsilon = 1e-6  # 수렴 기준

# 전이 확률과 보상 설정 (예시로 가정, 실제 환경에서는 다를 수 있음)
# T = np.zeros((16, 4, 16))  # 전이 확률 T(s, a, s')
# R = np.zeros((16, 4, 16))  # 보상 R(s, a, s')
# 4x4 FrozenLake 환경에서 상태와 행동의 개수
num_states = 16
num_actions = 4

# 전이 확률 초기화
T = np.zeros((num_states, num_actions, num_states))

# 보상 초기화
R = np.zeros((num_states, num_actions, num_states))

# 상태 전이 예시 (단순화된 형태로 일부만 보여줌)
# 상(0), 하(1), 좌(2), 우(3) 행동에 대한 전이 확률 설정
# 상(0) 행동
T[0, 0, 0] = 1  # 벽이므로 상태가 변하지 않음
T[1, 0, 1] = 1  # 벽이므로 상태가 변하지 않음
T[4, 0, 0] = 1  # (1, 0) -> (0, 0)
T[5, 0, 1] = 1  # (1, 1) -> (0, 1)

# 하(1) 행동
T[0, 1, 4] = 1  # (0, 0) -> (1, 0)
T[1, 1, 5] = 1  # (0, 1) -> (1, 1)
T[4, 1, 8] = 1  # (1, 0) -> (2, 0)
T[5, 1, 9] = 1  # (1, 1) -> (2, 1)

# 좌(2) 행동
T[0, 2, 0] = 1  # 벽이므로 상태가 변하지 않음
T[1, 2, 0] = 1  # (0, 1) -> (0, 0)
T[4, 2, 4] = 1  # 벽이므로 상태가 변하지 않음
T[5, 2, 4] = 1  # (1, 1) -> (1, 0)

# 우(3) 행동
T[0, 3, 1] = 1  # (0, 0) -> (0, 1)
T[1, 3, 2] = 1  # (0, 1) -> (0, 2)
T[4, 3, 5] = 1  # (1, 0) -> (1, 1)
T[5, 3, 6] = 1  # (1, 1) -> (1, 2)

# 보상 설정 예시
R[0, 1, 4] = 0  # (0, 0)에서 하로 이동하여 (1, 0)이 됨 -> 보상 0
R[0, 3, 1] = 0  # (0, 0)에서 우로 이동하여 (0, 1)이 됨 -> 보상 0
R[14, 1, 15] = 1  # 목표 지점에 도달하면 보상 1

print("전이 확률 T:")
print(T)

print("보상 R:")
print(R)

# 초기 가치 함수
V = np.zeros(16)

# 가치 반복
while True:
    delta = 0
    for s in states:
        v = V[s]
        V[s] = max(sum(T[s, a, s_prime] * (R[s, a, s_prime] + gamma * V[s_prime])
                       for s_prime in states) for a in actions)
        delta = max(delta, abs(v - V[s]))
    if delta < epsilon:
        break

print("최적 가치 함수 V:")
print(V)

전이 확률 T:
[[[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
보상 R:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 

### Q-learning 알고리즘을 사용하여 Q-값을 업데이트하는 간단한 예제

In [13]:
import numpy as np

# 환경 설정
num_states = 16  # 상태의 개수 (예: FrozenLake 4x4)
num_actions = 4  # 행동의 개수 (상, 하, 좌, 우)
gamma = 0.9  # 할인 인자
alpha = 0.1  # 학습률
epsilon = 0.1  # 탐색률
num_episodes = 1000  # 에피소드 수

# Q-테이블 초기화
Q = np.zeros((num_states, num_actions))

# 전이 확률과 보상 설정 (단순화된 예시)
T = np.zeros((num_states, num_actions, num_states))
R = np.zeros((num_states, num_actions, num_states))

# 환경 초기화 (상태 전이와 보상 설정)
# (자세한 설정은 환경에 따라 다를 수 있음)
for s in range(num_states):
    for a in range(num_actions):
        next_state = (s + a) % num_states  # 임의로 설정한 전이 규칙
        T[s, a, next_state] = 1
        R[s, a, next_state] = -1  # 임의의 보상

# Q-learning 알고리즘
for episode in range(num_episodes):
    state = np.random.randint(0, num_states)  # 초기 상태 무작위 설정
    done = False

    while not done:
        # 행동 선택 (탐욕적 선택과 탐색의 혼합)
        if np.random.rand() < epsilon:
            action = np.random.randint(0, num_actions)  # 탐색
        else:
            action = np.argmax(Q[state, :])  # 탐욕적 선택

        # 다음 상태 및 보상 관찰
        next_state = np.argmax(T[state, action, :])
        reward = R[state, action, next_state]

        # Q-값 업데이트
        best_next_action = np.argmax(Q[next_state, :])
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])

        state = next_state

        # 종료 조건 (예: 목표 상태에 도달하면 종료)
        if state == num_states - 1:
            done = True

print("최적의 Q-테이블:")
print(Q)

최적의 Q-테이블:
[[-4.46660087 -4.51176221 -4.4956603  -4.54851143]
 [-4.79659477 -4.76395001 -4.78297903 -4.79722439]
 [-4.83823706 -4.86310957 -4.82905312 -4.84320774]
 [-4.83510516 -4.81155304 -4.83434125 -4.81488675]
 [-4.73206297 -4.74794415 -4.71097079 -4.71399932]
 [-4.58070578 -4.63125618 -4.60550905 -4.61683806]
 [-4.44158527 -4.46110077 -4.42039216 -4.4195377 ]
 [-4.33801514 -4.32767603 -4.31381198 -4.30527794]
 [-4.18507352 -4.19033302 -4.16989379 -4.17930821]
 [-3.89047996 -3.94321715 -3.95811653 -3.89310765]
 [-3.80666914 -3.83524612 -3.80366033 -3.81289225]
 [-3.74168432 -3.71029271 -3.69993062 -3.71800141]
 [-3.35992596 -3.31463193 -3.31898454 -3.27461311]
 [-3.35497766 -3.2952054  -3.24451924 -3.24461855]
 [-3.26883405 -3.2450667  -3.40221574 -3.34475906]
 [-2.59793124 -2.67963711 -2.67502189 -2.63742641]]


### 시간차(Temporal Difference) 학습과 Q-Learning

In [14]:
import numpy as np

# 데이터 설정 및 초기화
nan = np.nan
T = np.array([
    [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
    [[0.0, 1.0, 0.0], [nan, nan, nan], [0.0, 0.0, 1.0]],
    [[nan, nan, nan], [0.8, 0.1, 0.1], [nan, nan, nan]],
])
R = np.array([
    [[10., 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]],
    [[0.0, 0.0, 0.0], [nan, nan, nan], [0.0, 0.0, -50]],
    [[nan, nan, nan], [40., 0.0, 0.0], [nan, nan, nan]],
])
possible_actions = [[0, 1, 2], [0, 2], [1]]

# 학습 파라미터 설정
learning_rate0 = 0.05
learning_rate_decay = 0.1
n_iterations = 20000
discount_factor = 0.99

s = 0

Q = np.full((3, 3), -np.inf)
for state, actions in enumerate(possible_actions):
    Q[state, actions] = 0.0

# Q-learning 알고리즘 적용
for iteration in range(n_iterations):
    a = np.random.choice(possible_actions[s])
    sp = np.random.choice(range(3), p=T[s, a])
    reward = R[s, a, sp]
    learning_rate = learning_rate0 / (1 + iteration * learning_rate_decay)
    Q[s, a] = (1 - learning_rate) * Q[s, a] + learning_rate * (reward + discount_factor * np.max(Q[sp]))
    s = sp

# 최적의 정책 계산
np.set_printoptions(precision=2)
print(Q)
optimal_policy = np.argmax(Q, axis=1)
print("Optimal Policy:", optimal_policy)


[[  5.52   1.87   1.48]
 [  0.     -inf -15.05]
 [  -inf  13.31   -inf]]
Optimal Policy: [0 0 1]


In [15]:
import gym
import numpy as np

env=gym.make('FrozenLake-v1',is_slippery=False) # 환경 생성
Q=np.zeros([env.observation_space.n,env.action_space.n]) # Q 배열 초기화

rho=0.8 # 학습률
lamda=0.99 # 할인율

n_episode=2000
length_episode=100

# 최적 행동 가치 함수 찾기
for i in range(n_episode):
    s=env.reset() # 새로운 에피소드 시작
    for j in range(length_episode):
        argmaxs=np.argwhere(Q[s,:]==np.amax(Q[s,:])).flatten().tolist()
        a=np.random.choice(argmaxs)
        s1,r,done,_=env.step(a)
        Q[s,a]=Q[s,a]+rho*(r+lamda*np.max(Q[s1,:])-Q[s,a]) # 식 (9.18)
        s=s1
        if done:
            break

np.set_printoptions(precision=2)
print(Q)

AttributeError: module 'numpy' has no attribute 'bool8'

In [17]:
import numpy as np
import random
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque

# 하이퍼 매개변수 설정
rho=0.9 # 학습률
lamda=0.99 # 할인율
eps=0.9
eps_decay=0.999
batch_siz=64
n_episode=100

# 신경망을 설계해주는 함수
def deep_network():
    mlp=Sequential()
    mlp.add(Dense(32,input_dim=env.observation_space.shape[0],activation='relu'))
    mlp.add(Dense(32,activation='relu'))
    mlp.add(Dense(env.action_space.n,activation='linear'))
    mlp.compile(loss='mse',optimizer='Adam')
    return mlp

# DQN 학습
def model_learning():
    mini_batch=np.asarray(random.sample(D,batch_siz))
    state=np.asarray([mini_batch[i,0] for i in range(batch_siz)])
    action=mini_batch[:,1]
    reward=mini_batch[:,2]
    state1=np.asarray([mini_batch[i,3] for i in range(batch_siz)])
    done=mini_batch[:,4]

    target=model.predict(state)
    target1=model.predict(state1)

    for i in range(batch_siz):
        if done[i]:
            target[i][action[i]]=reward[i]
        else:
            target[i][action[i]]+=rho*((reward[i]+lamda*np.amax(target1[i]))-target[i][action[i]]) # Q 러닝(식 (9.19))
    model.fit(state,target,batch_size=batch_siz,epochs=1,verbose=0)

env=gym.make("CartPole-v0")

model=deep_network() # 신경망 생성
D=deque(maxlen=2000) # 리플레이 메모리 초기화
scores=[]
max_steps=env.spec.max_episode_steps

# 신경망 학습
for i in range(n_episode):
    s=env.reset()
    long_reward=0

    while True:
        r=np.random.random()
        eps=max(0.01,eps*eps_decay) # 엡시론을 조금씩 줄여나감
        if(r<eps):
            a=np.random.randint(0,env.action_space.n) # 랜덤 정책
        else:
            q=model.predict(np.reshape(s,[1,4])) # 신경망이 예측한 행동
            a=np.argmax(q[0])
        s1,r,done,_=env.step(a)
        if done and long_reward<max_steps-1: # 실패
            r=-100

        D.append((s,a,r,s1,done))

        if len(D)>batch_siz*3:
            model_learning()

        s=s1
        long_reward+=r

        if done:
            long_reward=long_reward if long_reward==max_steps else long_reward+100
            print(i,"번째 에피소드의 점수:",long_reward)
            scores.append(long_reward)
            break

    if i>10 and np.mean(scores[-5:])>(0.95*max_steps):
        break

# 신경망 저장
model.save("./cartpole_by_DQN.h5")
env.close()

import matplotlib.pyplot as plt

plt.plot(range(1,len(scores)+1),scores)
plt.title('DQN scores for CartPole-v0')
plt.ylabel('Score')
plt.xlabel('Episode')
plt.grid()
plt.show()

c:\DEV\envs\py3_10_tf\lib\site-packages\gym\envs\registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
c:\DEV\envs\py3_10_tf\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\DEV\envs\py3_10_tf\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\DEV\envs\py3_10_tf\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(

AttributeError: module 'numpy' has no attribute 'bool8'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import numpy as np
import gym
import time

# 신경망 블러옴
model=load_model('./cartpole_by_DQN.h5')

env=gym.make("CartPole-v0")
long_reward=0

# CartPole 플레이
s=env.reset()
while True:
    q=model.predict(np.reshape(s,[1,4])) # 신경망이 예측한 행동
    a=np.argmax(q[0])
    s1,r,done,_=env.step(a)
    s=s1
    long_reward+=r

    env.render()
    time.sleep(0.02)

    if done:
        print("에피소드의 점수:",long_reward)
        break

env.close()